
### Ingestion pipeline for Lifion application logs

In [0]:
# Calculate the date range for the last 7 days
from datetime import datetime, timedelta


date_range = 7
end_date = datetime.now() - timedelta(days=1)
# Add in correlationid to join search and click, tracking from 3/18/2025
start_date = max(datetime(2025, 3, 18), end_date - timedelta(days=date_range))
date_range = (end_date - start_date).days
print(f"Start date: {start_date}")
print(f"End date: {end_date}")
print(f"Date range: {date_range}")

Start date: 2025-03-18 00:00:00
End date: 2025-03-20 01:25:38.442226
Date range: 2


In [0]:
for single_date in (start_date + timedelta(n) for n in range(date_range)):
    print(single_date)

2025-03-18 00:00:00
2025-03-19 00:00:00


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
# from ingestion.etl_on_logs import DQSelectionLogsLoader
# from utils.data_utils import DataUtils
import uuid
from datetime import datetime


### Need to get DQSelectionLogsLoader --> /delta/lifion_logs_for_sdq


### SDQ-get_logs-job run

In [0]:
# # DQSelectionLogsLoader

# import boto3
# from typing import Dict
# import multiprocessing
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import *
# from delta.tables import *
# import logging
# from logging import config

# # TODO: make this generic
# config.dictConfig({
#     'version': 1,
#     'formatters': {
#         'standard': {
#             'format': '%(levelname)-8s %(name)-12s %(message)s',
#         },
#     },
#     'handlers': {
#         'stream': {
#             'level': 'INFO',
#             'class': 'logging.StreamHandler',
#             'formatter': 'standard',
#         },
#     },
#     'loggers': {
#         '': {
#             'handlers': ['stream'],
#             'level': 'DEBUG',
#         },
#     },
# })
# logger = logging.getLogger(__name__)


# class DQSelectionLogsLoader(object):
    
#     def __init__(self, sparkSession: SparkSession, loggerObj, source_path = '/mnt/lifion-data-lake-logs/processed',
#     checkpoint_path = '/mnt/lifion_logs_checkpoint', table_name='DQSelectionLogsLoader',
#     roleArn='arn:aws:iam::760781926556:role/DatabricksMGMTAthenaRole') -> None:
#         """Class to perform reading from S3 backet of log files based on SNS queue event connected to the s3 bucket

#         Args:
#             sparkSession (SparkSession): spark session
#             loggerObj (_type_): logger, note: does not work
#             source_path (str, optional): Mounted drive with logs. Defaults to '/mnt/lifion-data-lake-logs/processed'.
#             checkpoint_path (str, optional): autoloader checkoint. Defaults to '/mnt/lifion_logs_checkpoint'.
#             table_name (str, optional): . Defaults to '/delta/lifion_logs_for_sdq'.
#             roleArn (str, optional): . Defaults to 'arn:aws:iam::760781926556:role/DatabricksMGMTAthenaRole'.
#         """    
#         self.spark = sparkSession
#         self.logger = logger.getChild(self.__class__.__name__)
#         self.region='us-east-1'
#         self.roleArn=roleArn
#         self.numFilesPerPartition = 36
#         self.source_path= source_path
#         self.checkpoint_path = checkpoint_path
#         self.table_name=table_name
#         # set the default conf
#         self.spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")
#         self.spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")
#         self.spark.conf.set("spark.sql.files.ignoreMissingFiles", "true")
#         self.spark.conf.set( 
#                "spark.sql.shuffle.partitions", self.numFilesPerPartition)    
#         self.num_cores = multiprocessing.cpu_count()
#         self.logger = loggerObj
#         self.logger.info(f"number of available cores {self.num_cores}")
        

    
#     def get_streaming_df(self):
#         self.logger.info(f"Starting execution of data import")  
#         df=(self.spark.readStream
#             .format("cloudFiles")
#             .option("cloudFiles.format", "parquet")
#             .option("cloudFiles.includeExistingFiles", "false")
#             .option("cloudFiles.schemaLocation", self.checkpoint_path)
#             .option("cloudFiles.inferColumnTypes", "true") 
#             .option("cloudFiles.region",self.region)
#             .option("cloudFiles.useNotifications", "true")
#             .option("cloudFiles.roleArn",self.roleArn)
#             .option("cloudFiles.schemaHints", "email STRING, amazon_trace_id STRING, duration STRING")
#             .load(self.source_path)
#             .selectExpr(  "env", "year", "month", "day", 
#                             "_timestamp", "client_id", "associate_id", 
#                             "trace_id", "session_id","page_load_id",
#                             "message:dqId as dq_id" ,    # extracts {"tags":{"page":{"name":...}}}
#                             "message:message::string as value", # extracts {"tags":{"page":{"id":...}}} and casts to int
#                             "message:cacheKey as cacheKey",     # extracts {"tags":{"eventType":...}}
#                             "message:executionTime::float as executionTime"
#                         )
#                         .where("image_name='data-query' and session_id is not Null and  page_load_id is not Null")    
#                         .where("value='Request Completed' and container_id like 'data-query-reader%'"))
            
#         return df

#     def write_to_delta(self, df):
#         if "." in self.table_name:
#             (df.writeStream
#                 .format("delta")
#                 .partitionBy("env", "year", "month", "day")
#                 .option("checkpointLocation", self.checkpoint_path)
#                 .trigger(availableNow=True)
#                 .outputMode("append")
#                 .toTable(self.table_name))     
   
#         else:    
#             (df.writeStream
#                 .format("delta")
#                 .partitionBy("env", "year", "month", "day")
#                 .option("checkpointLocation", self.checkpoint_path)
#                 .trigger(availableNow=True)
#                 .outputMode("append")
#                 .start(self.table_name))     



#     def read_from_table(self):
#         return self.spark.read.format("delta").load(self.table_name)        

#     def _optimize(self):
        
#         deltaTable = DeltaTable.forPath(self.spark, self.table_name )
#         deltaTable.optimize().executeCompaction()

In [0]:
# def get_job_metadata()->str:
#     """get run_id from databricks workflow run. used to query only records added in one particular run

#     Returns:
#         str: workflow run id or uuid if calling from the notebook
#     """    
#     import json
#     context_str = dbutils.notebook.entry_point.getDbutils().notebook().getContext().toJson()
#     context = json.loads(context_str)
#     run_id_obj = context.get('currentRunId', {})
#     #run_id = run_id_obj.get('id', None) if run_id_obj else None
#     job_id = context.get('tags', {}).get('jobId', None)
#     run_id = context.get('tags', {}).get('multitaskParentRunId', None)
#     if run_id ==None:
#         run_id=str(uuid.uuid4())
#     return (job_id, run_id) 

# job_metadata=get_job_metadata()

In [0]:
# logs_table = spark.read.json('sdq_test_data_2025-02-14.jsonl')

import pandas as pd

logs_table = pd.read_json("sdq_test_data_2025-02-14.jsonl", lines=True)

In [0]:
display(logs_table)

captureData,eventMetadata,eventVersion
"List(List(eyJkYXRhZnJhbWVfc3BsaXQiOnsiY29sdW1ucyI6WyJzZXNzaW9uX2lkIiwidGltZXN0YW1wIiwiY2xpZW50X2lkIiwiYXNzb2NpYXRlX2lkIiwidHJhY2VfaWQiLCJkcV9pZCIsInBhcmFtcyJdLCJkYXRhIjpbWyI0M2I1MmQ4NS0zMzhlLTQyMmEtYTM2NS1iYzkzYTYyNTlmZjEiLCJGcmksIDE0IEZlYiAyMDI1IDIxOjAwOjI0IEdNVCIsIjUyNjc0Mzc1LTBmMDctNDYzYy1iZTMwLTk4NGIyZmM5OTA3ZiIsIjg0YTJlZTZiLWEzZGMtNDNhYy1iZjY5LWRhN2NmYjU1ZGNhNCIsImY0ODhjNTgyLTk0YzgtNGI0My05MjcwLTU3MmZmMTk4MjAwMCIsImNkNmRkM2E0YjMyOTRhMTU5MmZkZDBhNzU5MGMwZmM4Iiwie1wiQXNzb2NpYXRlSURzXCI6W1wiODRhMmVlNmItYTNkYy00M2FjLWJmNjktZGE3Y2ZiNTVkY2E0XCJdfSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOSBHTVQiLCI1MjY3NDM3NS0wZjA3LTQ2M2MtYmUzMC05ODRiMmZjOTkwN2YiLCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTQiLCJkMTI0ZTlhMy1hZDU3LTQ5ODctOTlhMS1hMTU4ZjJmNDNjY2MiLCJjNjk2OGFmOTNkNDQ0NDdkYjlhMzQ0Yjk4YmMzZTkyOCIsIntcIkltcG9ydEpvYktleVwiOlwiNDVmM2EwZjUtMzIwOS00N2M4LWFmNDAtNmNkMGFlYWJkMTAxXCIsXCJyZXR1cm5Ub3RhbENvdW50T25seVwiOmZhbHNlfSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOSBHTVQiLCI1MjY3NDM3NS0wZjA3LTQ2M2MtYmUzMC05ODRiMmZjOTkwN2YiLCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTQiLCI2MmFjYTg3YS1jNTJkLTRlMDYtYmQ3My0wNjAxMWI4NjA1ZmEiLCJiMWE4ZGQ3ZWZkNWU0ODQxOWNiYjlkZGQ5NGM2ZmYyZSIsIntcImltcG9ydEpvYktleVwiOlwiNDVmM2EwZjUtMzIwOS00N2M4LWFmNDAtNmNkMGFlYWJkMTAxXCJ9Il0sWyI0M2I1MmQ4NS0zMzhlLTQyMmEtYTM2NS1iYzkzYTYyNTlmZjEiLCJGcmksIDE0IEZlYiAyMDI1IDIxOjAwOjA4IEdNVCIsIjUyNjc0Mzc1LTBmMDctNDYzYy1iZTMwLTk4NGIyZmM5OTA3ZiIsIjg0YTJlZTZiLWEzZGMtNDNhYy1iZjY5LWRhN2NmYjU1ZGNhNCIsIjAyODkyZDVkLTY0Y2YtNGY2ZS04YWI4LTEwYjkzZjA3YmU3ZiIsImQ0ODljMjkwMGUzMzQ2NjViMzg3Y2VhMWE1YzA1NzVlIiwie1wicmV0dXJuVG90YWxDb3VudE9ubHlcIjp0cnVlfSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOCBHTVQiLCI1MjY3NDM3NS0wZjA3LTQ2M2MtYmUzMC05ODRiMmZjOTkwN2YiLCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTQiLCJjZjU5ZTdkNy00NGU4LTQwN2EtOWY4Yi04OTg4YjY2YzA2MmUiLCI2MzdjYjA4NjRhNWE0YzhmODg4MmMyMzdmMTI3YTkzOCIsIntcInJldHVyblRvdGFsQ291bnRPbmx5XCI6dHJ1ZSxcInBhZ2luYXRpb25cIjp7XCJsaW1pdFwiOjEwMDAwMDAwLFwic2tpcFwiOjAsXCJyZXN1bHRXaXRoVG90YWxDb3VudFwiOnRydWV9fSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOCBHTVQiLCI1MjY3NDM3NS0wZjA3LTQ2M2MtYmUzMC05ODRiMmZjOTkwN2YiLCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTQiLCI2ZTU0NGQ4Ni00M2YxLTQyZGUtOWI4OS0zNjhkYTkzZDM2YjUiLCI2NTU2NDQ5ODViN2U0NTExYWM2YzRmY2IzMDUyNTA3NyIsIntcImltcG9ydEpvYktleVwiOlwiNDVmM2EwZjUtMzIwOS00N2M4LWFmNDAtNmNkMGFlYWJkMTAxXCJ9Il1dfX0=, BASE64, INPUT, application/json), List(eyJwcmVkaWN0aW9ucyI6IFtbImMyMDJhOTMzMjM2NTRhMTBiOGQwZjA2N2E2YjQwM2JifHtcInJldHVyblRvdGFsQ291bnRPbmx5XCI6ZmFsc2V9IiwgIkVNUFRZIiwgIkVNUFRZIiwgIkVNUFRZIiwgIkVNUFRZIl1dfQ==, BASE64, OUTPUT, application/json))","List(List({""traceId"":""f488c582-94c8-4b43-9270-572ff1982000""}), 2319d3a7-dbb3-4da2-8c19-6563f16ce75d, 734151c7-6ff7-408f-a513-f8e00ea2158d, 2025-02-14T21:00:24Z)",0
"List(List(eyJkYXRhZnJhbWVfc3BsaXQiOnsiY29sdW1ucyI6WyJzZXNzaW9uX2lkIiwidGltZXN0YW1wIiwiY2xpZW50X2lkIiwiYXNzb2NpYXRlX2lkIiwidHJhY2VfaWQiLCJkcV9pZCIsInBhcmFtcyJdLCJkYXRhIjpbWyI0M2I1MmQ4NS0zMzhlLTQyMmEtYTM2NS1iYzkzYTYyNTlmZjEiLCJGcmksIDE0IEZlYiAyMDI1IDIxOjAwOjI0IEdNVCIsIjUyNjc0Mzc1LTBmMDctNDYzYy1iZTMwLTk4NGIyZmM5OTA3ZiIsIjg0YTJlZTZiLWEzZGMtNDNhYy1iZjY5LWRhN2NmYjU1ZGNhNCIsIjRjZWRmODg5LTA4MGUtNDc4Mi1hMjczLThkMTdiNDRiYjk1NSIsImMyMDJhOTMzMjM2NTRhMTBiOGQwZjA2N2E2YjQwM2JiIiwie1wicmV0dXJuVG90YWxDb3VudE9ubHlcIjpmYWxzZX0iXSxbIjQzYjUyZDg1LTMzOGUtNDIyYS1hMzY1LWJjOTNhNjI1OWZmMSIsIkZyaSwgMTQgRmViIDIwMjUgMjE6MDA6MjQgR01UIiwiNTI2NzQzNzUtMGYwNy00NjNjLWJlMzAtOTg0YjJmYzk5MDdmIiwiODRhMmVlNmItYTNkYy00M2FjLWJmNjktZGE3Y2ZiNTVkY2E0IiwiZjQ4OGM1ODItOTRjOC00YjQzLTkyNzAtNTcyZmYxOTgyMDAwIiwiY2Q2ZGQzYTRiMzI5NGExNTkyZmRkMGE3NTkwYzBmYzgiLCJ7XCJBc3NvY2lhdGVJRHNcIjpbXCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTRcIl19Il0sWyI0M2I1MmQ4NS0zMzhlLTQyMmEtYTM2NS1iYzkzYTYyNTlmZjEiLCJGcmksIDE0IEZlYiAyMDI1IDIxOjAwOjA5IEdNVCIsIjU

In [0]:
import base64
import ast

sample = ast.literal_eval(base64.b64decode(logs_table.iloc[0]["captureData"]["endpointInput"]['data']).decode("utf-8"))

In [0]:
sample

{'dataframe_split': {'columns': ['session_id',
   'timestamp',
   'client_id',
   'associate_id',
   'trace_id',
   'dq_id',
   'params'],
  'data': [['43b52d85-338e-422a-a365-bc93a6259ff1',
    'Fri, 14 Feb 2025 21:00:24 GMT',
    '52674375-0f07-463c-be30-984b2fc9907f',
    '84a2ee6b-a3dc-43ac-bf69-da7cfb55dca4',
    'f488c582-94c8-4b43-9270-572ff1982000',
    'cd6dd3a4b3294a1592fdd0a7590c0fc8',
    '{"AssociateIDs":["84a2ee6b-a3dc-43ac-bf69-da7cfb55dca4"]}'],
   ['43b52d85-338e-422a-a365-bc93a6259ff1',
    'Fri, 14 Feb 2025 21:00:09 GMT',
    '52674375-0f07-463c-be30-984b2fc9907f',
    '84a2ee6b-a3dc-43ac-bf69-da7cfb55dca4',
    'd124e9a3-ad57-4987-99a1-a158f2f43ccc',
    'c6968af93d44447db9a344b98bc3e928',
    '{"ImportJobKey":"45f3a0f5-3209-47c8-af40-6cd0aeabd101","returnTotalCountOnly":false}'],
   ['43b52d85-338e-422a-a365-bc93a6259ff1',
    'Fri, 14 Feb 2025 21:00:09 GMT',
    '52674375-0f07-463c-be30-984b2fc9907f',
    '84a2ee6b-a3dc-43ac-bf69-da7cfb55dca4',
    '62aca87a-c52d-

In [0]:
import json

# def eval_sdq(encoded_list):
#     try: 
#         res = []
#         for x in encoded_list:
#             items = ast.literal_eval(base64.b64decode(x).decode('utf-8'))
#             df = pd.DataFrame(data=items['dataframe_split']["data"], 
#                             columns=items['dataframe_split']["columns"])
#             d = df.to_dict(orient="records")
#             for i in d:
#                 res.append(json.dumps(d[i]))
#     except Exception as e:
#         print(f"Error processing data: {e}")
#         return ["Error"]
#     return res
def eval_sdq(encoded_list):
        res = []
        for x in encoded_list:
            items = ast.literal_eval(base64.b64decode(x).decode('utf-8'))
            df = pd.DataFrame(data=items['dataframe_split']["data"], 
                            columns=items['dataframe_split']["columns"])
            d = df.to_dict(orient="records")
            for record in d:
                res.append(json.dumps(record))
        return res

In [0]:
sample = logs_table["captureData"].apply(lambda x: x["endpointInput"]["data"])

In [0]:
len(sample)

30

In [0]:
sample[0]

'eyJkYXRhZnJhbWVfc3BsaXQiOnsiY29sdW1ucyI6WyJzZXNzaW9uX2lkIiwidGltZXN0YW1wIiwiY2xpZW50X2lkIiwiYXNzb2NpYXRlX2lkIiwidHJhY2VfaWQiLCJkcV9pZCIsInBhcmFtcyJdLCJkYXRhIjpbWyI0M2I1MmQ4NS0zMzhlLTQyMmEtYTM2NS1iYzkzYTYyNTlmZjEiLCJGcmksIDE0IEZlYiAyMDI1IDIxOjAwOjI0IEdNVCIsIjUyNjc0Mzc1LTBmMDctNDYzYy1iZTMwLTk4NGIyZmM5OTA3ZiIsIjg0YTJlZTZiLWEzZGMtNDNhYy1iZjY5LWRhN2NmYjU1ZGNhNCIsImY0ODhjNTgyLTk0YzgtNGI0My05MjcwLTU3MmZmMTk4MjAwMCIsImNkNmRkM2E0YjMyOTRhMTU5MmZkZDBhNzU5MGMwZmM4Iiwie1wiQXNzb2NpYXRlSURzXCI6W1wiODRhMmVlNmItYTNkYy00M2FjLWJmNjktZGE3Y2ZiNTVkY2E0XCJdfSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOSBHTVQiLCI1MjY3NDM3NS0wZjA3LTQ2M2MtYmUzMC05ODRiMmZjOTkwN2YiLCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTQiLCJkMTI0ZTlhMy1hZDU3LTQ5ODctOTlhMS1hMTU4ZjJmNDNjY2MiLCJjNjk2OGFmOTNkNDQ0NDdkYjlhMzQ0Yjk4YmMzZTkyOCIsIntcIkltcG9ydEpvYktleVwiOlwiNDVmM2EwZjUtMzIwOS00N2M4LWFmNDAtNmNkMGFlYWJkMTAxXCIsXCJyZXR1cm5Ub3RhbENvdW50T25seVwiOmZhbHNlfSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWE

In [0]:
import pprint

pprint.pprint(eval_sdq(sample)[0])

('{"session_id": "43b52d85-338e-422a-a365-bc93a6259ff1", "timestamp": "Fri, 14 '
 'Feb 2025 21:00:24 GMT", "client_id": "52674375-0f07-463c-be30-984b2fc9907f", '
 '"associate_id": "84a2ee6b-a3dc-43ac-bf69-da7cfb55dca4", "trace_id": '
 '"f488c582-94c8-4b43-9270-572ff1982000", "dq_id": '
 '"cd6dd3a4b3294a1592fdd0a7590c0fc8", "params": '
 '"{\\"AssociateIDs\\":[\\"84a2ee6b-a3dc-43ac-bf69-da7cfb55dca4\\"]}"}')


In [0]:
df_sdq = spark.createDataFrame(logs_table["captureData"].apply(lambda x: x["endpointInput"]["data"]), ["encoded_list"])
display(df_sdq)

encoded_list
eyJkYXRhZnJhbWVfc3BsaXQiOnsiY29sdW1ucyI6WyJzZXNzaW9uX2lkIiwidGltZXN0YW1wIiwiY2xpZW50X2lkIiwiYXNzb2NpYXRlX2lkIiwidHJhY2VfaWQiLCJkcV9pZCIsInBhcmFtcyJdLCJkYXRhIjpbWyI0M2I1MmQ4NS0zMzhlLTQyMmEtYTM2NS1iYzkzYTYyNTlmZjEiLCJGcmksIDE0IEZlYiAyMDI1IDIxOjAwOjI0IEdNVCIsIjUyNjc0Mzc1LTBmMDctNDYzYy1iZTMwLTk4NGIyZmM5OTA3ZiIsIjg0YTJlZTZiLWEzZGMtNDNhYy1iZjY5LWRhN2NmYjU1ZGNhNCIsImY0ODhjNTgyLTk0YzgtNGI0My05MjcwLTU3MmZmMTk4MjAwMCIsImNkNmRkM2E0YjMyOTRhMTU5MmZkZDBhNzU5MGMwZmM4Iiwie1wiQXNzb2NpYXRlSURzXCI6W1wiODRhMmVlNmItYTNkYy00M2FjLWJmNjktZGE3Y2ZiNTVkY2E0XCJdfSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOSBHTVQiLCI1MjY3NDM3NS0wZjA3LTQ2M2MtYmUzMC05ODRiMmZjOTkwN2YiLCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTQiLCJkMTI0ZTlhMy1hZDU3LTQ5ODctOTlhMS1hMTU4ZjJmNDNjY2MiLCJjNjk2OGFmOTNkNDQ0NDdkYjlhMzQ0Yjk4YmMzZTkyOCIsIntcIkltcG9ydEpvYktleVwiOlwiNDVmM2EwZjUtMzIwOS00N2M4LWFmNDAtNmNkMGFlYWJkMTAxXCIsXCJyZXR1cm5Ub3RhbENvdW50T25seVwiOmZhbHNlfSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOSBHTVQiLCI1MjY3NDM3NS0wZjA3LTQ2M2MtYmUzMC05ODRiMmZjOTkwN2YiLCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTQiLCI2MmFjYTg3YS1jNTJkLTRlMDYtYmQ3My0wNjAxMWI4NjA1ZmEiLCJiMWE4ZGQ3ZWZkNWU0ODQxOWNiYjlkZGQ5NGM2ZmYyZSIsIntcImltcG9ydEpvYktleVwiOlwiNDVmM2EwZjUtMzIwOS00N2M4LWFmNDAtNmNkMGFlYWJkMTAxXCJ9Il0sWyI0M2I1MmQ4NS0zMzhlLTQyMmEtYTM2NS1iYzkzYTYyNTlmZjEiLCJGcmksIDE0IEZlYiAyMDI1IDIxOjAwOjA4IEdNVCIsIjUyNjc0Mzc1LTBmMDctNDYzYy1iZTMwLTk4NGIyZmM5OTA3ZiIsIjg0YTJlZTZiLWEzZGMtNDNhYy1iZjY5LWRhN2NmYjU1ZGNhNCIsIjAyODkyZDVkLTY0Y2YtNGY2ZS04YWI4LTEwYjkzZjA3YmU3ZiIsImQ0ODljMjkwMGUzMzQ2NjViMzg3Y2VhMWE1YzA1NzVlIiwie1wicmV0dXJuVG90YWxDb3VudE9ubHlcIjp0cnVlfSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOCBHTVQiLCI1MjY3NDM3NS0wZjA3LTQ2M2MtYmUzMC05ODRiMmZjOTkwN2YiLCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTQiLCJjZjU5ZTdkNy00NGU4LTQwN2EtOWY4Yi04OTg4YjY2YzA2MmUiLCI2MzdjYjA4NjRhNWE0YzhmODg4MmMyMzdmMTI3YTkzOCIsIntcInJldHVyblRvdGFsQ291bnRPbmx5XCI6dHJ1ZSxcInBhZ2luYXRpb25cIjp7XCJsaW1pdFwiOjEwMDAwMDAwLFwic2tpcFwiOjAsXCJyZXN1bHRXaXRoVG90YWxDb3VudFwiOnRydWV9fSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOCBHTVQiLCI1MjY3NDM3NS0wZjA3LTQ2M2MtYmUzMC05ODRiMmZjOTkwN2YiLCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTQiLCI2ZTU0NGQ4Ni00M2YxLTQyZGUtOWI4OS0zNjhkYTkzZDM2YjUiLCI2NTU2NDQ5ODViN2U0NTExYWM2YzRmY2IzMDUyNTA3NyIsIntcImltcG9ydEpvYktleVwiOlwiNDVmM2EwZjUtMzIwOS00N2M4LWFmNDAtNmNkMGFlYWJkMTAxXCJ9Il1dfX0=
eyJkYXRhZnJhbWVfc3BsaXQiOnsiY29sdW1ucyI6WyJzZXNzaW9uX2lkIiwidGltZXN0YW1wIiwiY2xpZW50X2lkIiwiYXNzb2NpYXRlX2lkIiwidHJhY2VfaWQiLCJkcV9pZCIsInBhcmFtcyJdLCJkYXRhIjpbWyI0M2I1MmQ4NS0zMzhlLTQyMmEtYTM2NS1iYzkzYTYyNTlmZjEiLCJGcmksIDE0IEZlYiAyMDI1IDIxOjAwOjI0IEdNVCIsIjUyNjc0Mzc1LTBmMDctNDYzYy1iZTMwLTk4NGIyZmM5OTA3ZiIsIjg0YTJlZTZiLWEzZGMtNDNhYy1iZjY5LWRhN2NmYjU1ZGNhNCIsIjRjZWRmODg5LTA4MGUtNDc4Mi1hMjczLThkMTdiNDRiYjk1NSIsImMyMDJhOTMzMjM2NTRhMTBiOGQwZjA2N2E2YjQwM2JiIiwie1wicmV0dXJuVG90YWxDb3VudE9ubHlcIjpmYWxzZX0iXSxbIjQzYjUyZDg1LTMzOGUtNDIyYS1hMzY1LWJjOTNhNjI1OWZmMSIsIkZyaSwgMTQgRmViIDIwMjUgMjE6MDA6MjQgR01UIiwiNTI2NzQzNzUtMGYwNy00NjNjLWJlMzAtOTg0YjJmYzk5MDdmIiwiODRhMmVlNmItYTNkYy00M2FjLWJmNjktZGE3Y2ZiNTVkY2E0IiwiZjQ4OGM1ODItOTRjOC00YjQzLTkyNzAtNTcyZmYxOTgyMDAwIiwiY2Q2ZGQzYTRiMzI5NGExNTkyZmRkMGE3NTkwYzBmYzgiLCJ7XCJBc3NvY2lhdGVJRHNcIjpbXCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTRcIl19Il0sWyI0M2I1MmQ4NS0zMzhlLTQyMmEtYTM2NS1iYzkzYTYyNTlmZjEiLCJGcmksIDE0IEZlYiAyMDI1IDIxOjAwOjA5IEdNVCIsIjUyNjc0Mzc1LTBmMDctNDYzYy1iZTMwLTk4NGIyZmM5OTA3ZiIsIjg0YTJlZTZiLWEzZGMtNDNhYy1iZjY5LWRhN2NmYjU1ZGNhNCIsImQxMjRlOWEzLWFkNTctNDk4Ny05OWExLWExNThmMmY0M2NjYyIsImM2OTY4YWY5M2Q0NDQ0N2RiOWEzNDRiOThiYzNlOTI4Iiwie1wiSW1wb3J0Sm9iS2V5XCI6XCI0NWYzYTBmNS0zMjA5LTQ3YzgtYWY0MC02Y2QwYWVhYmQxMDFcIixcInJldHVyblRvdGFsQ291bnRPbmx5XCI6ZmFsc2V9Il0sWyI0M2I1MmQ4NS0zMzhlLTQyMmEtYTM2NS1iYzkzYTYyNTlmZjEiLCJGcmksIDE0IEZlYiAyMDI1IDIxOjAwOjA5IEdNVCIsIjUyNjc0Mzc1LTBmMDctNDYzYy1iZTMwLTk4NGIyZm

In [0]:
%sql
CREATE OR REPLACE FUNCTION onedata_us_east_1_shared_dit.nas_raw_lyric_search_dit.literal_eval_sdq_demo(x STRING) 
RETURNS ARRAY<STRING>
LANGUAGE PYTHON
AS $$
import ast
import json
import base64
import pandas as pd

def eval_sdq(encoded_list):
    res = []
    try: 
        items = ast.literal_eval(base64.b64decode(encoded_list).decode('utf-8'))
        df = pd.DataFrame(data=items['dataframe_split']["data"], 
                        columns=items['dataframe_split']["columns"])
        d = df.to_dict(orient="records")
        for record in d:
            res.append(json.dumps(record))
        return res
    except Exception as e:
        print(f"Error processing data: {e}")
        return ["Error"]
        
return eval_sdq(x)
$$

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType
import pyspark.sql.functions as F
from pyspark.sql.functions import expr

# Define the schema for the JSON objects
schema = StructType([
                StructField('session_id', StringType(), nullable=False), 
                StructField('timestamp',  StringType(), nullable=False),
                StructField('client_id',  StringType(), nullable=False),
                StructField('associate_id',  StringType(), nullable=False),
                StructField('trace_id',  StringType(), nullable=False),
                StructField('dq_id',  StringType(), nullable=False),
                StructField('params',  StringType(), nullable=False)
])

# Explode the list of JSON objects
res = df_sdq.withColumn("encoded_list_extract_sdq_demo", expr("onedata_us_east_1_shared_dit.nas_raw_lyric_search_dit.literal_eval_sdq_demo(encoded_list)"))
exploded_df = res.withColumn("decoded_list", F.explode(F.col("encoded_list_extract_sdq_demo")))
parsed_df = exploded_df.withColumn("decoded_list", F.from_json(F.col("decoded_list"), schema))
final_df = parsed_df.select(*[col for col in res.columns if col != "encoded_list_extract_sdq_demo"], "decoded_list.*")

In [0]:
display(final_df)

encoded_list,session_id,timestamp,client_id,associate_id,trace_id,dq_id,params
eyJkYXRhZnJhbWVfc3BsaXQiOnsiY29sdW1ucyI6WyJzZXNzaW9uX2lkIiwidGltZXN0YW1wIiwiY2xpZW50X2lkIiwiYXNzb2NpYXRlX2lkIiwidHJhY2VfaWQiLCJkcV9pZCIsInBhcmFtcyJdLCJkYXRhIjpbWyI0M2I1MmQ4NS0zMzhlLTQyMmEtYTM2NS1iYzkzYTYyNTlmZjEiLCJGcmksIDE0IEZlYiAyMDI1IDIxOjAwOjI0IEdNVCIsIjUyNjc0Mzc1LTBmMDctNDYzYy1iZTMwLTk4NGIyZmM5OTA3ZiIsIjg0YTJlZTZiLWEzZGMtNDNhYy1iZjY5LWRhN2NmYjU1ZGNhNCIsImY0ODhjNTgyLTk0YzgtNGI0My05MjcwLTU3MmZmMTk4MjAwMCIsImNkNmRkM2E0YjMyOTRhMTU5MmZkZDBhNzU5MGMwZmM4Iiwie1wiQXNzb2NpYXRlSURzXCI6W1wiODRhMmVlNmItYTNkYy00M2FjLWJmNjktZGE3Y2ZiNTVkY2E0XCJdfSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOSBHTVQiLCI1MjY3NDM3NS0wZjA3LTQ2M2MtYmUzMC05ODRiMmZjOTkwN2YiLCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTQiLCJkMTI0ZTlhMy1hZDU3LTQ5ODctOTlhMS1hMTU4ZjJmNDNjY2MiLCJjNjk2OGFmOTNkNDQ0NDdkYjlhMzQ0Yjk4YmMzZTkyOCIsIntcIkltcG9ydEpvYktleVwiOlwiNDVmM2EwZjUtMzIwOS00N2M4LWFmNDAtNmNkMGFlYWJkMTAxXCIsXCJyZXR1cm5Ub3RhbENvdW50T25seVwiOmZhbHNlfSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOSBHTVQiLCI1MjY3NDM3NS0wZjA3LTQ2M2MtYmUzMC05ODRiMmZjOTkwN2YiLCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTQiLCI2MmFjYTg3YS1jNTJkLTRlMDYtYmQ3My0wNjAxMWI4NjA1ZmEiLCJiMWE4ZGQ3ZWZkNWU0ODQxOWNiYjlkZGQ5NGM2ZmYyZSIsIntcImltcG9ydEpvYktleVwiOlwiNDVmM2EwZjUtMzIwOS00N2M4LWFmNDAtNmNkMGFlYWJkMTAxXCJ9Il0sWyI0M2I1MmQ4NS0zMzhlLTQyMmEtYTM2NS1iYzkzYTYyNTlmZjEiLCJGcmksIDE0IEZlYiAyMDI1IDIxOjAwOjA4IEdNVCIsIjUyNjc0Mzc1LTBmMDctNDYzYy1iZTMwLTk4NGIyZmM5OTA3ZiIsIjg0YTJlZTZiLWEzZGMtNDNhYy1iZjY5LWRhN2NmYjU1ZGNhNCIsIjAyODkyZDVkLTY0Y2YtNGY2ZS04YWI4LTEwYjkzZjA3YmU3ZiIsImQ0ODljMjkwMGUzMzQ2NjViMzg3Y2VhMWE1YzA1NzVlIiwie1wicmV0dXJuVG90YWxDb3VudE9ubHlcIjp0cnVlfSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOCBHTVQiLCI1MjY3NDM3NS0wZjA3LTQ2M2MtYmUzMC05ODRiMmZjOTkwN2YiLCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTQiLCJjZjU5ZTdkNy00NGU4LTQwN2EtOWY4Yi04OTg4YjY2YzA2MmUiLCI2MzdjYjA4NjRhNWE0YzhmODg4MmMyMzdmMTI3YTkzOCIsIntcInJldHVyblRvdGFsQ291bnRPbmx5XCI6dHJ1ZSxcInBhZ2luYXRpb25cIjp7XCJsaW1pdFwiOjEwMDAwMDAwLFwic2tpcFwiOjAsXCJyZXN1bHRXaXRoVG90YWxDb3VudFwiOnRydWV9fSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOCBHTVQiLCI1MjY3NDM3NS0wZjA3LTQ2M2MtYmUzMC05ODRiMmZjOTkwN2YiLCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTQiLCI2ZTU0NGQ4Ni00M2YxLTQyZGUtOWI4OS0zNjhkYTkzZDM2YjUiLCI2NTU2NDQ5ODViN2U0NTExYWM2YzRmY2IzMDUyNTA3NyIsIntcImltcG9ydEpvYktleVwiOlwiNDVmM2EwZjUtMzIwOS00N2M4LWFmNDAtNmNkMGFlYWJkMTAxXCJ9Il1dfX0=,43b52d85-338e-422a-a365-bc93a6259ff1,"Fri, 14 Feb 2025 21:00:24 GMT",52674375-0f07-463c-be30-984b2fc9907f,84a2ee6b-a3dc-43ac-bf69-da7cfb55dca4,f488c582-94c8-4b43-9270-572ff1982000,cd6dd3a4b3294a1592fdd0a7590c0fc8,"{""AssociateIDs"":[""84a2ee6b-a3dc-43ac-bf69-da7cfb55dca4""]}"
eyJkYXRhZnJhbWVfc3BsaXQiOnsiY29sdW1ucyI6WyJzZXNzaW9uX2lkIiwidGltZXN0YW1wIiwiY2xpZW50X2lkIiwiYXNzb2NpYXRlX2lkIiwidHJhY2VfaWQiLCJkcV9pZCIsInBhcmFtcyJdLCJkYXRhIjpbWyI0M2I1MmQ4NS0zMzhlLTQyMmEtYTM2NS1iYzkzYTYyNTlmZjEiLCJGcmksIDE0IEZlYiAyMDI1IDIxOjAwOjI0IEdNVCIsIjUyNjc0Mzc1LTBmMDctNDYzYy1iZTMwLTk4NGIyZmM5OTA3ZiIsIjg0YTJlZTZiLWEzZGMtNDNhYy1iZjY5LWRhN2NmYjU1ZGNhNCIsImY0ODhjNTgyLTk0YzgtNGI0My05MjcwLTU3MmZmMTk4MjAwMCIsImNkNmRkM2E0YjMyOTRhMTU5MmZkZDBhNzU5MGMwZmM4Iiwie1wiQXNzb2NpYXRlSURzXCI6W1wiODRhMmVlNmItYTNkYy00M2FjLWJmNjktZGE3Y2ZiNTVkY2E0XCJdfSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOSBHTVQiLCI1MjY3NDM3NS0wZjA3LTQ2M2MtYmUzMC05ODRiMmZjOTkwN2YiLCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTQiLCJkMTI0ZTlhMy1hZDU3LTQ5ODctOTlhMS1hMTU4ZjJmNDNjY2MiLCJjNjk2OGFmOTNkNDQ0NDdkYjlhMzQ0Yjk4YmMzZTkyOCIsIntcIkltcG9ydEpvYktleVwiOlwiNDVmM2EwZjUtMzIwOS00N2M4LWFmNDAtNmNkMGFlYWJkMTAxXCIsXCJyZXR1cm5Ub3RhbENvdW50T25seVwiOmZhbHNlfSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOSBHTVQiLCI1MjY3NDM3NS0wZjA3LTQ2M2MtYmUzMC05ODRiMmZjOTk

In [0]:
from pyspark.sql import functions as F

yesterday = spark.sql("SELECT date_sub(current_date(), 1) AS yesterday").collect()[0]['yesterday']

df = spark.createDataFrame([(yesterday,)], ["date"])
display(df)

df = df.withColumn("year", F.year(F.lit(yesterday)))




date
2025-02-16



### SDQ Data Ingestion

In [0]:
logs_table=spark.read.table("nas_raw_lyric_search_dit.ling_test_sdq")

In [0]:
display(logs_table)

encoded_list,session_id,timestamp,client_id,associate_id,trace_id,dq_id,params,year,month,day
eyJkYXRhZnJhbWVfc3BsaXQiOnsiY29sdW1ucyI6WyJzZXNzaW9uX2lkIiwidGltZXN0YW1wIiwiY2xpZW50X2lkIiwiYXNzb2NpYXRlX2lkIiwidHJhY2VfaWQiLCJkcV9pZCIsInBhcmFtcyJdLCJkYXRhIjpbWyI0M2I1MmQ4NS0zMzhlLTQyMmEtYTM2NS1iYzkzYTYyNTlmZjEiLCJGcmksIDE0IEZlYiAyMDI1IDIxOjAwOjI0IEdNVCIsIjUyNjc0Mzc1LTBmMDctNDYzYy1iZTMwLTk4NGIyZmM5OTA3ZiIsIjg0YTJlZTZiLWEzZGMtNDNhYy1iZjY5LWRhN2NmYjU1ZGNhNCIsImY0ODhjNTgyLTk0YzgtNGI0My05MjcwLTU3MmZmMTk4MjAwMCIsImNkNmRkM2E0YjMyOTRhMTU5MmZkZDBhNzU5MGMwZmM4Iiwie1wiQXNzb2NpYXRlSURzXCI6W1wiODRhMmVlNmItYTNkYy00M2FjLWJmNjktZGE3Y2ZiNTVkY2E0XCJdfSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOSBHTVQiLCI1MjY3NDM3NS0wZjA3LTQ2M2MtYmUzMC05ODRiMmZjOTkwN2YiLCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTQiLCJkMTI0ZTlhMy1hZDU3LTQ5ODctOTlhMS1hMTU4ZjJmNDNjY2MiLCJjNjk2OGFmOTNkNDQ0NDdkYjlhMzQ0Yjk4YmMzZTkyOCIsIntcIkltcG9ydEpvYktleVwiOlwiNDVmM2EwZjUtMzIwOS00N2M4LWFmNDAtNmNkMGFlYWJkMTAxXCIsXCJyZXR1cm5Ub3RhbENvdW50T25seVwiOmZhbHNlfSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOSBHTVQiLCI1MjY3NDM3NS0wZjA3LTQ2M2MtYmUzMC05ODRiMmZjOTkwN2YiLCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTQiLCI2MmFjYTg3YS1jNTJkLTRlMDYtYmQ3My0wNjAxMWI4NjA1ZmEiLCJiMWE4ZGQ3ZWZkNWU0ODQxOWNiYjlkZGQ5NGM2ZmYyZSIsIntcImltcG9ydEpvYktleVwiOlwiNDVmM2EwZjUtMzIwOS00N2M4LWFmNDAtNmNkMGFlYWJkMTAxXCJ9Il0sWyI0M2I1MmQ4NS0zMzhlLTQyMmEtYTM2NS1iYzkzYTYyNTlmZjEiLCJGcmksIDE0IEZlYiAyMDI1IDIxOjAwOjA4IEdNVCIsIjUyNjc0Mzc1LTBmMDctNDYzYy1iZTMwLTk4NGIyZmM5OTA3ZiIsIjg0YTJlZTZiLWEzZGMtNDNhYy1iZjY5LWRhN2NmYjU1ZGNhNCIsIjAyODkyZDVkLTY0Y2YtNGY2ZS04YWI4LTEwYjkzZjA3YmU3ZiIsImQ0ODljMjkwMGUzMzQ2NjViMzg3Y2VhMWE1YzA1NzVlIiwie1wicmV0dXJuVG90YWxDb3VudE9ubHlcIjp0cnVlfSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOCBHTVQiLCI1MjY3NDM3NS0wZjA3LTQ2M2MtYmUzMC05ODRiMmZjOTkwN2YiLCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTQiLCJjZjU5ZTdkNy00NGU4LTQwN2EtOWY4Yi04OTg4YjY2YzA2MmUiLCI2MzdjYjA4NjRhNWE0YzhmODg4MmMyMzdmMTI3YTkzOCIsIntcInJldHVyblRvdGFsQ291bnRPbmx5XCI6dHJ1ZSxcInBhZ2luYXRpb25cIjp7XCJsaW1pdFwiOjEwMDAwMDAwLFwic2tpcFwiOjAsXCJyZXN1bHRXaXRoVG90YWxDb3VudFwiOnRydWV9fSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOCBHTVQiLCI1MjY3NDM3NS0wZjA3LTQ2M2MtYmUzMC05ODRiMmZjOTkwN2YiLCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTQiLCI2ZTU0NGQ4Ni00M2YxLTQyZGUtOWI4OS0zNjhkYTkzZDM2YjUiLCI2NTU2NDQ5ODViN2U0NTExYWM2YzRmY2IzMDUyNTA3NyIsIntcImltcG9ydEpvYktleVwiOlwiNDVmM2EwZjUtMzIwOS00N2M4LWFmNDAtNmNkMGFlYWJkMTAxXCJ9Il1dfX0=,43b52d85-338e-422a-a365-bc93a6259ff1,"Fri, 14 Feb 2025 21:00:24 GMT",52674375-0f07-463c-be30-984b2fc9907f,84a2ee6b-a3dc-43ac-bf69-da7cfb55dca4,f488c582-94c8-4b43-9270-572ff1982000,cd6dd3a4b3294a1592fdd0a7590c0fc8,"{""AssociateIDs"":[""84a2ee6b-a3dc-43ac-bf69-da7cfb55dca4""]}",2025,2025,2025
eyJkYXRhZnJhbWVfc3BsaXQiOnsiY29sdW1ucyI6WyJzZXNzaW9uX2lkIiwidGltZXN0YW1wIiwiY2xpZW50X2lkIiwiYXNzb2NpYXRlX2lkIiwidHJhY2VfaWQiLCJkcV9pZCIsInBhcmFtcyJdLCJkYXRhIjpbWyI0M2I1MmQ4NS0zMzhlLTQyMmEtYTM2NS1iYzkzYTYyNTlmZjEiLCJGcmksIDE0IEZlYiAyMDI1IDIxOjAwOjI0IEdNVCIsIjUyNjc0Mzc1LTBmMDctNDYzYy1iZTMwLTk4NGIyZmM5OTA3ZiIsIjg0YTJlZTZiLWEzZGMtNDNhYy1iZjY5LWRhN2NmYjU1ZGNhNCIsImY0ODhjNTgyLTk0YzgtNGI0My05MjcwLTU3MmZmMTk4MjAwMCIsImNkNmRkM2E0YjMyOTRhMTU5MmZkZDBhNzU5MGMwZmM4Iiwie1wiQXNzb2NpYXRlSURzXCI6W1wiODRhMmVlNmItYTNkYy00M2FjLWJmNjktZGE3Y2ZiNTVkY2E0XCJdfSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOSBHTVQiLCI1MjY3NDM3NS0wZjA3LTQ2M2MtYmUzMC05ODRiMmZjOTkwN2YiLCI4NGEyZWU2Yi1hM2RjLTQzYWMtYmY2OS1kYTdjZmI1NWRjYTQiLCJkMTI0ZTlhMy1hZDU3LTQ5ODctOTlhMS1hMTU4ZjJmNDNjY2MiLCJjNjk2OGFmOTNkNDQ0NDdkYjlhMzQ0Yjk4YmMzZTkyOCIsIntcIkltcG9ydEpvYktleVwiOlwiNDVmM2EwZjUtMzIwOS00N2M4LWFmNDAtNmNkMGFlYWJkMTAxXCIsXCJyZXR1cm5Ub3RhbENvdW50T25seVwiOmZhbHNlfSJdLFsiNDNiNTJkODUtMzM4ZS00MjJhLWEzNjUtYmM5M2E2MjU5ZmYxIiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMDowOSBHTVQiLCI1MjY3NDM3NS0wZ


### SDQ Data Reader

In [0]:
 %pip install mlflow pandas awswrangler humanize

Looking in indexes: https://artifactory.us.caas.oneadp.com/artifactory/api/pypi/pypi/simple/
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
# SageMaker config yaml
# ---
# int:
#   group_id: int-use1
#   account_number: '784001157278'
#   subnets: subnet-0ed63db467d06a823,subnet-00960ae016f4f6b4f,subnet-069d9d6273785424f
#   security_group: sg-00a84b97154c615f0
#   databrick_env: dit
# appbuild:
#   group_id: appbuild-02
#   account_number: '334703519168'
#   subnets: subnet-0431184dcdac89191,subnet-00aefaf3def52f1eb,subnet-0c2b91c9fc2a562a8
#   security_group: sg-0e45ef727d0420f92
#   databrick_env: dit
# uat:
#   group_id: uat-03
#   account_number: '945644374226'
#   databrick_env: fit
#   subnets: subnet-02a7f5864d0f100aa,subnet-0ae99dc321a2f9698,subnet-024b94f48f56289cd
#   security_group: sg-0085a0f818a95e84e
# prep:
#   group_id: prep-use1
#   account_number: '515302890279'
#   subnets: subnet-01fb88e8c26db9a34,subnet-0a8205b3bbfb5dc71,subnet-07dcd6f840ce1b06d
#   security_group: sg-07ab8fc5c0336d619
#   databrick_env: iat
# prod:
#   group_id: prod-use1
#   account_number: '084558427856'
#   subnets: subnet-0a8ce77a43dd579f5,subnet-06b9793d246dfdaee,subnet-04c042740f0d6e189
#   security_group: sg-02d70fdbf664e525a
#   databrick_env: prod

In [0]:
# def get_job_metadata()->str:
#     """get run_id from databricks workflow run. used to query only records added in one particular run

#     Returns:
#         str: workflow run id or uuid if calling from the notebook
#     """    
#     import json
#     context_str = dbutils.notebook.entry_point.getDbutils().notebook().getContext().toJson()
#     context = json.loads(context_str)
#     run_id_obj = context.get('currentRunId', {})
#     #run_id = run_id_obj.get('id', None) if run_id_obj else None
#     job_id = context.get('tags', {}).get('jobId', None)
#     run_id = context.get('tags', {}).get('multitaskParentRunId', None)
#     if run_id ==None:
#         run_id=str(uuid.uuid4())
#     return (job_id, run_id) 

# job_metadata=get_job_metadata() 

# if job_metadata[0] != None:
#     group_id = dbutils.widgets.get("group_id")
#     env = dbutils.widgets.get("env_name")
#     adp_env = dbutils.widgets.get('adp_env')
#     start = dbutils.widgets.get("start_date")
#     end = dbutils.widgets.get("end_date")
#     total_days = dbutils.widgets.get("total_days")
#     test_hours = int(dbutils.widgets.get("test_hours"))
#     delta_db = dbutils.widgets.get("delta_db")
#     table_name = dbutils.widgets.get('cacheble_dq_table_name')
#     clients = dbutils.widgets.get('clients')
#     canary_test = dbutils.widgets.get('canary_test')
# else:
#     group_id = "prod-use1"
#     env = "prod"
#     adp_env = 'prod'
#     start = "04/13/23"
#     end = "04/18/23"
#     total_days = "3"
#     test_hours = 2
#     delta_db = "nas_raw_lifion_prod"
#     table_name = "lifion_cacheable_dq"
#     clients = "e9df3133-53c1-46c7-8679-04c8e1f8bf13,11610030-82f0-4ecb-ad7f-3191b89d6c48"
#     canary_test = "True"

# if env == 'uat':
#     lifion_group_id = 'uat-03'
# else:
#     lifion_group_id = group_id

group_id = "prod-use1"
env = "prod"
adp_env = 'prod'
start = "04/13/23"
end = "04/18/23"
total_days = "3"
test_hours = 2
delta_db = "nas_raw_lifion_prod"
table_name = "lifion_cacheable_dq"
clients = "e9df3133-53c1-46c7-8679-04c8e1f8bf13,11610030-82f0-4ecb-ad7f-3191b89d6c48"
canary_test = "True"
lifion_group_id = group_id

In [0]:
test_clients = {
    'Capstone Logistics Training': '74203693-75b5-4433-8da3-b988f0fd94b0',
    'PCS QA Client': 'e9df3133-53c1-46c7-8679-04c8e1f8bf13',
    'Leadership Demo Client 01': '67eafe6f-af3a-492e-8862-0f616fc6cdeb',
    'ADP Demo Manufacturing': '4de71952-94aa-4802-bd32-6d981860395c',
    'Demo Client': '0887ddbb-e94c-4859-83a8-5346dea683c3',
    'Demo Client 01': '3846339d-4e69-49dc-95cb-32c878c2d01b',
    'GNC': '7d817782-8538-4762-8dcf-ad981874afe8',
    'GNC Training': '47f0a59e-25ae-45c3-83a2-c475ecf0e4d7',
}

In [0]:
test_clients_2 = {y: x for x, y in test_clients.items()}
test_clients_2

{'74203693-75b5-4433-8da3-b988f0fd94b0': 'Capstone Logistics Training',
 'e9df3133-53c1-46c7-8679-04c8e1f8bf13': 'PCS QA Client',
 '67eafe6f-af3a-492e-8862-0f616fc6cdeb': 'Leadership Demo Client 01',
 '4de71952-94aa-4802-bd32-6d981860395c': 'ADP Demo Manufacturing',
 '0887ddbb-e94c-4859-83a8-5346dea683c3': 'Demo Client',
 '3846339d-4e69-49dc-95cb-32c878c2d01b': 'Demo Client 01',
 '7d817782-8538-4762-8dcf-ad981874afe8': 'GNC',
 '47f0a59e-25ae-45c3-83a2-c475ecf0e4d7': 'GNC Training'}

In [0]:
client_ids = clients.split(",")
print(f"Number of clients: {len(client_ids)}")

Number of clients: 2


In [0]:
sql_cliend_ids = str(tuple([key for key in client_ids])).replace(',)', ')')
sql_cliend_ids

"('e9df3133-53c1-46c7-8679-04c8e1f8bf13', '11610030-82f0-4ecb-ad7f-3191b89d6c48')"

In [0]:
sql_cliend_ids = ('74203693-75b5-4433-8da3-b988f0fd94b0')

In [0]:
 ## Utilize Spark to collect logs ingested from Sagemaker Data Capture and apply some transformations.

In [0]:
# dbfs:/databricks/mlflow-tracking/1190548181184099/2904c7c2ef254992aad1ae14ecdec8f8/artifacts/model/conda.yaml

target_environment="int"
deployment_name="ml--smart-dq"
model_name="lifion--ml--smart-dq--prod-use1"
model_stage="production"
variant_name = "prod-variant"
instance_type="ml.m5.xlarge"
instance_count=1
auto_scaling=True
min_instances= 1
max_instances= 2
SageMakerVariantInvocationsPerInstanceScalingValue=700
containerStartupHealthCheckTimeoutInSeconds=900


dq_only_data_table = f"{delta_db}.lifion_sdq_params_{group_id}".replace("-", "_")
kibana_table_name = f"{delta_db}.lifion_logs_for_sdq"


#### Create training and test datasets from Sagemaker Data Capture logs
Utilize Spark to collect logs ingested from Sagemaker Data Capture and apply some transformations.

In [0]:
sdq_logs = spark.sql(f"select * from nas_raw_lyric_search_dit.ling_test_sdq")

In [0]:
sdq_logs.printSchema()

root
 |-- encoded_list: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- client_id: string (nullable = true)
 |-- associate_id: string (nullable = true)
 |-- trace_id: string (nullable = true)
 |-- dq_id: string (nullable = true)
 |-- params: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)



In [0]:
# https://adpdc-onedata-share1-dev-us-east.cloud.databricks.com/explore/data/models/onedata_us_east_1_shared_models_dev/onedata_ml_model_dev/lifion--ml--smart-dq/version/2?o=5954045386522247
# session_id
# client_id
# timestamp
# associate_id
# trace_id
# dq_id
# params


### Load models

In [0]:
import os
import mlflow

import json
from mlflow import MlflowClient

In [0]:
# onedata_us_east_1_shared_models_dev.onedata_ml_model_dev.lifion--ml--smart-dq

catalog_name = "onedata_us_east_1_shared_models_dev"
schema_name = "onedata_ml_model_dev"
model_name = "lifion--ml--smart-dq"
version = "2"

In [0]:
MODEL_NAME = catalog_name + "." + schema_name + "." + model_name

In [0]:
registry_url = "databricks-uc"
mlflow.set_registry_uri(registry_url)
client = MlflowClient()
model_uri = f"models:/{MODEL_NAME}/{version}"

In [0]:
def get_type(val):
    if val == "A list of non-empty values":
        return "list"
    if isinstance(val, bool):
        return "bool"
    if isinstance(val, int):
        return "int"
    if isinstance(val, float):
        return "float"
    try:
        val = datetime.strptime(val, "%Y-%m-%dT%H:%M:%S.%fZ")
        return "datetime"
    except (ValueError, TypeError) as exc:
        pass

    return "str"

In [0]:
import json
from pyspark.sql.functions import to_timestamp, col, concat, lit, asc
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

DataFrame[key: string, value: string]

In [0]:
fmt = 'EEE, dd MMM yyyy HH:mm:ss zzz'

dq_only_data_table = ""
sdq_logs = spark.read.format("delta").load(dq_only_data_table)

In [0]:
from pyspark.sql.functions import to_timestamp, col, concat, lit, asc
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
fmt = 'EEE, dd MMM yyyy HH:mm:ss zzz'
sdq_logs = (
           sdq_logs
        #    .where(partition)
            .withColumn("dq_id_label", concat(col("dq_id"),lit('|'),col("params")))
            # .withColumn("timestamp",to_timestamp(col("timestamp"),fmt))
            # .where(col('timestamp').between(start_time, now))
            # .where(f"client_id in {sql_cliend_ids}")
            .distinct()
           )

In [0]:
display(sdq_logs)

encoded_list,session_id,timestamp,client_id,associate_id,trace_id,dq_id,params,year,month,day,dq_id_label
eyJkYXRhZnJhbWVfc3BsaXQiOnsiY29sdW1ucyI6WyJzZXNzaW9uX2lkIiwidGltZXN0YW1wIiwiY2xpZW50X2lkIiwiYXNzb2NpYXRlX2lkIiwidHJhY2VfaWQiLCJkcV9pZCIsInBhcmFtcyJdLCJkYXRhIjpbWyJkMmNhNjg1ZS1iZjVmLTRkZGQtYjAyMS1lZmY4NWUzOWIzNjgiLCJGcmksIDE0IEZlYiAyMDI1IDIxOjAxOjA1IEdNVCIsIjkxNDQxM2RkLTliYjAtNDBmYS04YWE2LTdjNGViZWQwYmQyOCIsIjZjMmRkODJmLTE0NjEtNDlkOS05MjI1LTg4MjQ3MTFlZTYyZCIsIjAzNGUyNWJhLWQ2NGEtNGZhYi1iOGQxLWQ3OWY1N2E0NTAxNiIsImZiYzdkOThiODcyNjRmYWJhMjRjNmVjMTljYTBhYTIyIiwie1wibGltaXRcIjoyLFwiaW5fYXNzb2NpYXRlS2V5XCI6XCJkM2ViYzA1ZC04ZmNiLTQ4MzQtYWYzYi05MGY1M2EyMzhkYzhcIixcInJldHVyblRvdGFsQ291bnRPbmx5XCI6ZmFsc2UsXCJpbl9jYXRlZ29yeVwiOlwiTFRQVFwiLFwiaW5faW5jbHVkZV9mdXR1cmVcIjp0cnVlfSJdLFsiZDJjYTY4NWUtYmY1Zi00ZGRkLWIwMjEtZWZmODVlMzliMzY4IiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMTowNCBHTVQiLCI5MTQ0MTNkZC05YmIwLTQwZmEtOGFhNi03YzRlYmVkMGJkMjgiLCI2YzJkZDgyZi0xNDYxLTQ5ZDktOTIyNS04ODI0NzExZWU2MmQiLCJhOWViNmZjNS0yNzBlLTRlZDktYmNhOC03ZTNiZmY2N2M4MTciLCIyNTA4NTEyYzQ0ZWM0NmUxOGFmNmE4N2Y4YmQyY2RkZCIsIntcIkFzc29jaWF0ZUtleVwiOlwiZDNlYmMwNWQtOGZjYi00ODM0LWFmM2ItOTBmNTNhMjM4ZGM4XCIsXCJBc3NvY2lhdGVJRFwiOlwiZDNlYmMwNWQtOGZjYi00ODM0LWFmM2ItOTBmNTNhMjM4ZGM4XCIsXCJMb2dnZWRJbkFzc29jaWF0ZUlEXCI6XCI2YzJkZDgyZi0xNDYxLTQ5ZDktOTIyNS04ODI0NzExZWU2MmRcIixcIlNob3dDYW5jZWxsZWRFdmVudFwiOnRydWUsXCJXYWdJRFwiOm51bGx9Il0sWyJkMmNhNjg1ZS1iZjVmLTRkZGQtYjAyMS1lZmY4NWUzOWIzNjgiLCJGcmksIDE0IEZlYiAyMDI1IDIxOjAxOjA0IEdNVCIsIjkxNDQxM2RkLTliYjAtNDBmYS04YWE2LTdjNGViZWQwYmQyOCIsIjZjMmRkODJmLTE0NjEtNDlkOS05MjI1LTg4MjQ3MTFlZTYyZCIsImE5ZWI2ZmM1LTI3MGUtNGVkOS1iY2E4LTdlM2JmZjY3YzgxNyIsIjYxYWYyMGY2MzA3NzQ4ZjI5ZTg5ZTlkYzA5ODIyNGZlIiwie1wiQXNzb2NpYXRlS2V5XCI6XCJkM2ViYzA1ZC04ZmNiLTQ4MzQtYWYzYi05MGY1M2EyMzhkYzhcIixcIkFzc29jaWF0ZUlEXCI6XCJkM2ViYzA1ZC04ZmNiLTQ4MzQtYWYzYi05MGY1M2EyMzhkYzhcIixcIkxvZ2dlZEluQXNzb2NpYXRlSURcIjpcIjZjMmRkODJmLTE0NjEtNDlkOS05MjI1LTg4MjQ3MTFlZTYyZFwiLFwiU2hvd0NhbmNlbGxlZEV2ZW50XCI6dHJ1ZSxcIldhZ0lEXCI6bnVsbH0iXSxbImQyY2E2ODVlLWJmNWYtNGRkZC1iMDIxLWVmZjg1ZTM5YjM2OCIsIkZyaSwgMTQgRmViIDIwMjUgMjE6MDE6MDQgR01UIiwiOTE0NDEzZGQtOWJiMC00MGZhLThhYTYtN2M0ZWJlZDBiZDI4IiwiNmMyZGQ4MmYtMTQ2MS00OWQ5LTkyMjUtODgyNDcxMWVlNjJkIiwiYTllYjZmYzUtMjcwZS00ZWQ5LWJjYTgtN2UzYmZmNjdjODE3IiwiMGQ2ZmQxMTMyMGIwNGRkOWFmYzdmMDcxYjYxNDA5NDEiLCJ7XCJBc3NvY2lhdGVLZXlcIjpcImQzZWJjMDVkLThmY2ItNDgzNC1hZjNiLTkwZjUzYTIzOGRjOFwiLFwiQXNzb2NpYXRlSURcIjpcImQzZWJjMDVkLThmY2ItNDgzNC1hZjNiLTkwZjUzYTIzOGRjOFwiLFwiTG9nZ2VkSW5Bc3NvY2lhdGVJRFwiOlwiNmMyZGQ4MmYtMTQ2MS00OWQ5LTkyMjUtODgyNDcxMWVlNjJkXCIsXCJTaG93Q2FuY2VsbGVkRXZlbnRcIjp0cnVlLFwiV2FnSURcIjpudWxsfSJdLFsiZDJjYTY4NWUtYmY1Zi00ZGRkLWIwMjEtZWZmODVlMzliMzY4IiwiRnJpLCAxNCBGZWIgMjAyNSAyMTowMTowMSBHTVQiLCI5MTQ0MTNkZC05YmIwLTQwZmEtOGFhNi03YzRlYmVkMGJkMjgiLCI2YzJkZDgyZi0xNDYxLTQ5ZDktOTIyNS04ODI0NzExZWU2MmQiLCI1ZTljNWUxMS00YzE1LTRlNzYtOTU1ZC1jOWUwMDFmNjcwMzciLCIyNDJhZDlkYWI2MWQ0NjU2OGNhOTE0ZDFhOGUwMmNlMSIsIntcIkNvbmZpZ1NldHRpbmdOYW1lXCI6XCJEaXNwbGF5TmV3V29ya0RldGFpbHNUaWxlX1YzXCJ9Il0sWyJkMmNhNjg1ZS1iZjVmLTRkZGQtYjAyMS1lZmY4NWUzOWIzNjgiLCJGcmksIDE0IEZlYiAyMDI1IDIxOjAwOjU3IEdNVCIsIjkxNDQxM2RkLTliYjAtNDBmYS04YWE2LTdjNGViZWQwYmQyOCIsIjZjMmRkODJmLTE0NjEtNDlkOS05MjI1LTg4MjQ3MTFlZTYyZCIsIjYxNDJlODQwLWY0YjAtNGMyOS05OTZhLWM5NDVkNmRiNzZiYiIsIjQ0YTc3ZmNhYTY0NjQ3Mjg5MmEzNGQ3Y2NlMGI3NmRlIiwie1wiaW5wdXRfYXNzb2NpYXRlX2lkXCI6XCJkM2ViYzA1ZC04ZmNiLTQ4MzQtYWYzYi05MGY1M2EyMzhkYzhcIixcInJldHVyblRvdGFsQ291bnRPbmx5XCI6ZmFsc2V9Il1dfX0=,d2ca685e-bf5f-4ddd-b021-eff85e39b368,2025-02-14T21:01:04Z,914413dd-9bb0-40fa-8aa6-7c4ebed0bd28,6c2dd82f-1461-49d9-9225-8824711ee62d,a9eb6fc5-270e-4ed9-bca8-7e3bff67c817,0d6fd11320b04dd9afc7f071b6140941,"{""AssociateKey"":""d3ebc05d-8fcb-4834-af3b-90f53a238dc8"",""AssociateID"":""d3ebc05d-8fcb-4834-af3b-90f53a238dc8"",""LoggedInAssociateID"":""6c2dd82f-1461-49d9-9225-8824711ee62d"",""ShowCancelledEvent"":true,""WagID"":null}",2025,2025,2025,"0d6fd11320b04dd9afc7f071b6140941|{""AssociateKey"":""d3ebc05d-8fcb-4834-af3b-90f53a238dc8"",""AssociateID"":""d3ebc05d-8fcb-4834-af3b-90f53a238dc8"",""L

In [0]:
columns_to_drop = ['year', 'month', 'day']
sdq_logs = sdq_logs.drop(*columns_to_drop)

In [0]:
import json

def get_type(val):
    if val == "A list of non-empty values":
        return "list"
    if isinstance(val, bool):
        return "bool"
    if isinstance(val, int):
        return "int"
    if isinstance(val, float):
        return "float"
    try:
        val = datetime.strptime(val, "%Y-%m-%dT%H:%M:%S.%fZ")
        return "datetime"
    except (ValueError, TypeError) as exc:
        pass

    return "str"

df_params = sdq_logs.toPandas()
df_params=df_params.assign(
        param_types=df_params.params.apply(json.loads).apply(lambda x: ', '.join(sorted(list([f"{k}:{get_type(v)}"for k,v in x.items()])))),
    )

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-5129370582998504>, line 22
     18     return "str"
     20 df_params = sdq_logs.toPandas()
     21 df_params=df_params.assign(
---> 22         param_types=df_params.params.apply(json.loads).apply(lambda x: ', '.join(sorted(list([f"{k}:{get_type(v)}"for k,v in x.items()])))),
     23     )

File /databricks/python/lib/python3.11/site-packages/pandas/core/series.py:4771, in Series.apply(self, func, convert_dtype, args, **kwargs)
   4661 def apply(
   4662     self,
   4663     func: AggFuncType,
   (...)
   4666     **kwargs,
   4667 ) -> DataFrame | Series:
   4668     """
   4669     Invoke function on values of Series.
   4670 
   (...)
   4769     dtype: float64
   4770     """
-> 4771     return SeriesApply(self, func, convert_dtype, args, kwargs).apply()

File /databricks/python/lib/python3.11/site-packages/pandas/cor

In [0]:
import mlflow.sagemaker
import boto3
import time
import uuid
import mlflow
from mlflow.deployments import get_deploy_client
from mlflow.exceptions import MlflowException
import botocore
import yaml
from pathlib import Path
import os
from pyspark.dbutils import DBUtils
from pyspark.sql import SparkSession
import numpy as np
from mlflow.tracking import MlflowClient
import pprint
import time
spark = SparkSession.builder.getOrCreate()
dbutils = DBUtils(spark)


class ModelDeployer(object):
    def read_yaml(self, file_path):
        with open(file_path, "r") as f:
            return yaml.safe_load(f)
    
    def __init__(self,target_environment="dev",deployment_name="ml--smart-dq",
                 variant_name="prod-variant",instance_type="ml.m5.xlarge",instance_count=1, 
                 **kwargs):
        self.target_environment=target_environment
        self.variant_name=variant_name
        # self.globalConfig=self.read_yaml(f'{os.getcwd()}/inference/sagemaker_config.yaml')
        self.instance_type=instance_type
        self.instance_count=instance_count
        self.containerStartupHealthCheckTimeoutInSeconds=kwargs["containerStartupHealthCheckTimeoutInSeconds"]
        # self.environment = self.globalConfig[target_environment]
        self.environment = {
              "group_id": "int-use1",
              "account_number": '784001157278',
              "subnets": "subnet-0ed63db467d06a823,subnet-00960ae016f4f6b4f,subnet-069d9d6273785424f",
              "security_group": "sg-00a84b97154c615f0",
              "databrick_env": "dit"
        }
        # self.deployment_name=f'{deployment_name}--{self.environment["group_id"]}'
        self.deployment_name="ml--smart-dq--appbuild-02"
        self.tags = self.add_tags()
        
        # dbutils.widgets.text("tracking_scope", defaultValue=f'{self.environment["databrick_env"]}-nas-lifion_ml-sdq:cmr')
        # tracking_scope = dbutils.widgets.get('tracking_scope')
        # tracking_scope = "prod-nas-lifion_ml-sdq:cmr"
        # tracking_uri = f'databricks://{tracking_scope}'
        # print(f"tracking_uri: {tracking_uri}")
        # mlflow.set_tracking_uri(tracking_uri)

        # registry_scope = f"{self.environment['databrick_env']}-nas-lifion_ml-sdq:cmr"
        # registry_uri = f'databricks://{registry_scope}'
        registry_uri = "databricks-uc"
        mlflow.set_registry_uri(registry_uri)
        print(f"registry_uri: {registry_uri}")
        self.mlFlowClient = MlflowClient()
        
    #     # MGMT_ACCOUNT_ID= kwargs.get('mgmt_account_id',"123447371741")
    #     MGMT_ACCOUNT_ID = "334703519168"
    #     # self.AWS_REGION = kwargs.get("aws_region","us-east-1")
    #     self.AWS_REGION = "us-east-1"
    #     self.TIMEOUT_SECONDS=900
    #     self.ENABLE_CAPTURE = True
    #     self.DATA_CAPTURE_INITIALSAMPLINGPERCENTAGE=100
    #     self.IMAGE_ECR_URL = f'{MGMT_ACCOUNT_ID}.dkr.ecr.us-east-1.amazonaws.com/lifion/ml/mlflow-pyfunc:2.0.1'
    #     if target_environment =='prod':
    #          self.bucket=f'sagemaker-mlflow-prod-{MGMT_ACCOUNT_ID}'
    #     else:    
    #         self.bucket=f'sagemaker-mlflow-nonprod-{MGMT_ACCOUNT_ID}'
        
    #     # if target_environment =='uat':
    #     #     self.data_capture_bucket=f's3://ml-models-bucket-{self.environment["group_id"]}-prod/sdq'
    #     # else:
    #     #     self.data_capture_bucket=f's3://ml-models-bucket-{self.environment["group_id"]}/sdq'
    #     self.data_capture_bucket=f's3://ml-models-bucket-/sdq'
    #     self.assume_role_arn = f'arn:aws:iam::{self.environment["account_number"]}:role/MLCDO_SageMakerExecutionRole'
    #     self.execution_role_arn = f'arn:aws:iam::{self.environment["account_number"]}:role/MLCDO_SageMakerExecutionRole'
        
    #     self.sage_client=self.get_sagemaker_client()
    #     self.mode = mlflow.sagemaker.DEPLOYMENT_MODE_CREATE

    
    def add_tags(self):
        tags=[
                {
                    'Key':'Workstream',
                    'Value': 'ml'
                },
                {
                    'Key':'OwnerDetails',
                    'Value': 'Primary DL:LDP.ML@ADP.com'
                },
                {
                    'Key':'ApplicationTier',
                    'Value': 'web'
                },
                {
                    'Key': 'Application',
                    'Value': 'Lifion',
                },
                {
                    'Key': 'group_id',
                    'Value': self.environment['group_id']
                },
                {
                    'Key': 'Environment',
                    'Value': self.target_environment
                },
                {
                    'Key': 'Name',
                    'Value': self.deployment_name
                },
                ]
            
        return tags  

    # def create_sagemaker_config(self, mode):
    #     # Configuration
        
    #     vpc_config = {
    #             "SecurityGroupIds": [
    #                 self.environment['security_group']
    #             ],
    #             "Subnets": self.environment['subnets'].split(',')
    #         }
    #     data_capture_config = {
    #         'EnableCapture': self.ENABLE_CAPTURE,
    #         'InitialSamplingPercentage': self.DATA_CAPTURE_INITIALSAMPLINGPERCENTAGE,
    #         'DestinationS3Uri': self.data_capture_bucket,
    #         'CaptureOptions': [
    #             {'CaptureMode': 'Input'},
    #             {'CaptureMode': 'Output'}
    #         ]
    #     }

    #     config=dict(
    #         assume_role_arn=self.assume_role_arn,
    #         execution_role_arn=self.execution_role_arn,
    #         bucket=self.bucket,
    #         image_url=self.IMAGE_ECR_URL,
    #         region_name= self.AWS_REGION,
    #         mode=mode,
    #         archive=True,
    #         instance_type=self.instance_type,
    #         instance_count=self.instance_count,
    #         ContainerStartupHealthCheckTimeoutInSeconds=self.containerStartupHealthCheckTimeoutInSeconds,
    #         synchronous=True,
    #         timeout_seconds=self.TIMEOUT_SECONDS,
    #         variant_name=self.variant_name,
    #         vpc_config=vpc_config,
    #         data_capture_config=data_capture_config
    #     )
    #     return config
    
    # def add_models_tags(self, mv):
    #     for m in mv:
    #         #run once
    #         self.tags.append({
    #                     'Key': 'latest_version',
    #                     'Value': m.version
    #                 })
    #         self.tags.append({
    #                     'Key': 'run_id',
    #                     'Value': m.run_id
    #                 })
    #         self.tags.append({
    #                     'Key': 'current_stage',
    #                     'Value': m.current_stage
    #                 })
    #         break        
            

    # def get_sagemaker_client(self):
    #     sts_client = boto3.client('sts')
    #     # Call the assume_role method of the STSConnection object and pass the role
    #     # ARN and a role session name.
    #     assumed_role_object=sts_client.assume_role(
    #     RoleArn=self.assume_role_arn,
    #     RoleSessionName="AssumeRoleSession1")
    #     credentials=assumed_role_object['Credentials']
    #     sage_client = boto3.client('sagemaker', region_name=self.AWS_REGION, aws_access_key_id=credentials['AccessKeyId'],
    #     aws_secret_access_key=credentials['SecretAccessKey'],
    #     aws_session_token=credentials['SessionToken'],)
    #     return sage_client

    # def endpoint_exists(self):
    #     endpoint_exists=False
    #     endpoints=self.sage_client.list_endpoints()
    #     for endpoint in endpoints['Endpoints']:
    #         if endpoint['EndpointName'] == self.deployment_name:
    #            endpoint_exists=True
    #            print((f'deployment {self.deployment_name} exist, will perform update'))
    #     return endpoint_exists    
            


    # def _check_status(self):
    
    #     endpoint_description = self.sage_client.describe_endpoint(EndpointName=self.deployment_name)
    #     endpoint_status = endpoint_description["EndpointStatus"]
    #     return endpoint_status
    
    # def get_endpoint_config(self):
    #     endpoint_description = self.sage_client.describe_endpoint(EndpointName=self.deployment_name)
    #     endpointConfigName=endpoint_description['EndpointConfigName']
    #     endpoint_description = self.sage_client.describe_endpoint_config(EndpointConfigName=endpointConfigName)
    #     return endpoint_description
        
    # def _get_endpoint_arn(self, app_name):
    #     response = self.sage_client.describe_endpoint(EndpointName=self.deployment_name) 
    #     return response['EndpointArn']
    
    # def deploy_endpoint(self, model_name, model_stage):
    #     deploy_client = get_deploy_client(f'sagemaker:/us-east-1/{self.assume_role_arn}')
    #     model_uri=f"models:/{model_name}/{model_stage}"
    #     pprint.pprint(f"deploying model {model_uri}")
    #     pprint.pprint("Endpoint configuration:")
    #     if self.endpoint_exists():
    #         config=self.create_sagemaker_config(mlflow.sagemaker.DEPLOYMENT_MODE_REPLACE)
    #         pprint.pprint(config)
            
    #         try:
    #             #first deployment will use dafault containerStartupHealthCheckTimeoutInSeconds
    #             #if it fails, do with the updated deployment
    #             deploy_client.update_deployment(
    #                 self.deployment_name,
    #                 flavor="python_function",
    #                 model_uri=model_uri,
    #                 config=config)
    #         except MlflowException as ex:
    #             pprint.pprint(ex)
    #             self.update_config_and_endpoint()
    #     else:
    #         config=self.create_sagemaker_config(mlflow.sagemaker.DEPLOYMENT_MODE_CREATE)
    #         pprint.pprint(config)
    #         deploy_client.create_deployment(
    #         self.deployment_name,
    #         flavor="python_function",
    #         model_uri=model_uri,
    #         config=config)
            
    #     status = self._check_status()
    #     while status == 'Updating' or status =='Creating':
    #         time.sleep(20)
    #         status = self._check_status()
    #         self.add_models_tags(self.mlFlowClient.get_latest_versions(model_name,
    #             stages=[model_stage]))
    #     for resource in self.get_sagemaker_resource_arns():
    #          self.sage_client.add_tags(ResourceArn=resource,Tags=self.tags)
    #     pprint.pprint("tags to apply") 
    #     pprint.pprint(self.tags)   
    #     return status

    # def delete_endpoint(self):
    #     self.sage_client.delete_endpoint(EndpointName=self.deployment_name)

    # def update_config_and_endpoint(self):
    #     sm_describe=self.sage_client.describe_endpoint(EndpointName=self.deployment_name)
    #     sm_config_name=sm_describe['EndpointConfigName'] 
    #     response_config = self.sage_client.describe_endpoint_config(
    #         EndpointConfigName=sm_config_name)
    #     pvariant=response_config["ProductionVariants"][0]
    #     data_capture=response_config["DataCaptureConfig"]
    #     # based on the current config, create a new one
    #     endpointConfigName=f"{self.deployment_name}-{str(uuid.uuid4())[:8]}-u"
    #     response = self.sage_client.create_endpoint_config(
    #         EndpointConfigName=endpointConfigName,
    #         ProductionVariants=[
    #             {
    #                 'VariantName': pvariant["VariantName"],
    #                 'ModelName': pvariant["ModelName"],
    #                 'InitialInstanceCount': pvariant["InitialInstanceCount"],
    #                 'InstanceType': pvariant["InstanceType"],
    #                 'InitialVariantWeight': pvariant["InitialVariantWeight"],
    #                 'ContainerStartupHealthCheckTimeoutInSeconds': self.containerStartupHealthCheckTimeoutInSeconds
    #             },
    #             ],
    #         DataCaptureConfig={
    #                 'EnableCapture': data_capture["EnableCapture"],
    #                 'InitialSamplingPercentage': data_capture["InitialSamplingPercentage"],
    #                 'DestinationS3Uri': data_capture["DestinationS3Uri"],
    #                 'CaptureOptions': [
    #                     {'CaptureMode': 'Input'},
    #                     {'CaptureMode': 'Output'}
    #                 ]
    #             },
    #         Tags=self.tags
    #             )
    #     pprint.pprint(f"Updated Endpoint config created: {response}")
    #     response=self.sage_client.update_endpoint(
    #             EndpointName=self.deployment_name,
    #             EndpointConfigName=endpointConfigName,
    #             RetainAllVariantProperties=True,
    #             RetainDeploymentConfig=True
    #         )
    #     status = self._check_status()
    #     while status == 'Updating' or status =='Creating':
    #         time.sleep(20)
    #         status = self._check_status()

    #     pprint.pprint(f"Endpoint updated: {response}")
    #     return response

    # def get_sagemaker_resource_arns(self): 
    #     resource_arn=[]
    #     resource_arn.append(self._get_endpoint_arn(self.deployment_name))
    #     sm_describe=self.sage_client.describe_endpoint(EndpointName=self.deployment_name) 
    #     sm_config_name=sm_describe['EndpointConfigName']
    #     sm_config_arn=f'arn:aws:sagemaker:{self.AWS_REGION}:{self.environment["account_number"]}:endpoint-config/{sm_config_name}'
    #     resource_arn.append(sm_config_arn)
    #     sm_config_describe=self.sage_client.describe_endpoint_config(EndpointConfigName=sm_describe['EndpointConfigName'])  
    #     model_name=""
    #     for variant in sm_config_describe['ProductionVariants']:
    #         if variant['VariantName']=='prod-variant':
    #             model_name=variant['ModelName']
    #             model_arn=f'arn:aws:sagemaker:{self.AWS_REGION}:{self.environment["account_number"]}:model/{model_name}'    
    #             resource_arn.append(model_arn)
    #     return resource_arn

In [0]:
import boto3
from botocore.exceptions import ClientError
import json
import os
import yaml
from botocore.config import Config
import logging
from logging import config


config.dictConfig({
    'version': 1,
    'formatters': {
        'standard': {
            'format': '%(levelname)-8s %(name)-12s %(message)s',
        },
    },
    'handlers': {
        'stream': {
            'level': 'INFO',
            'class': 'logging.StreamHandler',
            'formatter': 'standard',
        },
    },
    'loggers': {
        '': {
            'handlers': ['stream'],
            'level': 'DEBUG',
        },
    },
})
logger = logging.getLogger(__name__)
Scalable_Dimension="sagemaker:variant:DesiredInstanceCount"

class AutoScalingWrapper:
    """Encapsulates Amazon EC2 Auto Scaling actions."""
    def __init__(self, target_environment, model_name, deployment_name, variant_name,
                 **kwargs):
        """
        :param autoscaling_client: A Boto3 Amazon EC2 Auto Scaling client.
        """
        self.region_name=kwargs.get("aws_region","us-east-1")
        self.auto_scaling=kwargs["auto_scaling"]
        self.min_instances= kwargs["min_instances"]
        self.max_instances= kwargs["max_instances"]
        self.SageMakerVariantInvocationsPerInstanceScalingValue=kwargs["SageMakerVariantInvocationsPerInstanceScalingValue"]
        self.target_environment=target_environment
        self.globalConfig=self.read_yaml(f'{os.getcwd()}/inference/sagemaker_config.yaml')
        self.environment = self.globalConfig[target_environment]
        self.assume_role_arn = f'arn:aws:iam::{self.environment["account_number"]}:role/MLCDO_SageMakerExecutionRole'
        self.execution_role_arn = f'arn:aws:iam::{self.environment["account_number"]}:role/MLCDO_SageMakerExecutionRole'
        self.client = self.get_autoscaling_client()
        self.service_name="sagemaker"
        endpoint_name=f'{deployment_name}--{self.environment["group_id"]}'
        self.ResourceId=f"endpoint/{endpoint_name}/variant/{variant_name}"
        self.PolicyType="TargetTrackingScaling"
        

    def apply_scaling_policy(self, PolicyName: str="prod_autoscaling",
            PolicyConfigFile: str ="scaling_policy.json") -> None:
        """Apply scaling policy to the endpoint

        Args:
            PolicyName (str, optional):policy name. Defaults to "prod_autoscaling".
            PolicyConfigFile (str, optional): path to the policy config file. Defaults to "scaling_policy.json".
        """            
        if self.auto_scaling:    
            try:
                policy=self.read_json(f'{os.getcwd()}/inference/{PolicyConfigFile}')
                # update the scaling policy
                policy["TargetValue"]=self.SageMakerVariantInvocationsPerInstanceScalingValue
                logger.info("policy to apply")
                logger.info(policy)
                response = self.client.put_scaling_policy(
                    PolicyName=PolicyName,
                    PolicyType=self.PolicyType,
                    ResourceId=self.ResourceId,
                    ScalableDimension=Scalable_Dimension,
                    ServiceNamespace=self.service_name,
                    TargetTrackingScalingPolicyConfiguration=policy)
                
            except ClientError as err:
                logger.error(
                    "Couldn't perform autoscaling. Here's why: %s: %s",
                    err.response['Error']['Code'], err.response['Error']['Message'])
                raise
        else:
            logger.info("Autoscaling policy did not apply because it is not configured in sagemaker_config.yaml") 
            logger.info("please set auto_scaling: True in this file")    
    
    def deregister_scalable_endpoint(self):
        """Deregisters an Application Auto Scaling scalable Sagemaker endpoint . 
        """        
        if self.auto_scaling:

            try:
                response = self.client.deregister_scalable_target(
                    ServiceNamespace=self.service_name,
                    ResourceId=self.ResourceId,
                    ScalableDimension=Scalable_Dimension)
                logger.info(json.dumps(response))    
            except ClientError as err:
                logger.error(
                    "Couldn't perform deregistering. Here's why: %s: %s",
                    err.response['Error']['Code'], err.response['Error']['Message'])
                raise 
        else:
            logger.info("can not deregister because autoscaling is not configured in sagemaker_config.yaml") 
            logger.info("please set auto_scaling: True in this file")  

    def register_scalable_endpoint(self):
        """Register endpoint with min and max instanses
        """        
        if self.auto_scaling:
            try:
                response = self.client.register_scalable_target(
                    ServiceNamespace=self.service_name,
                    ResourceId=self.ResourceId,
                    ScalableDimension=Scalable_Dimension,
                    MinCapacity=self.min_instances,
                    MaxCapacity=self.max_instances,
                    RoleARN=self.execution_role_arn)
                logger.info("Register endpoint with min and max instanses")
                logger.info(json.dumps(response))
            except ClientError as err:
                logger.error(
                    "Couldn't perform registration. Here's why: %s: %s",
                    err.response['Error']['Code'], err.response['Error']['Message'])
                raise 
        else:
            logger.info("can't register because autoscaling is not configured in sagemaker_config.yaml") 
            logger.info("please set auto_scaling: True in this file")      
    
    def read_yaml(self, file_path):      
        with open(file_path, "r") as f:
            return yaml.safe_load(f) 
    
    def read_json(self, file_path):
        with open(file_path, "r") as f:
            return json.load(f) 

    def get_autoscaling_client(self):
        """use sts and get the client with assuming role

        Returns:
            _type_: boto3 application-autoscaling client 
        """        
        sts_client = boto3.client('sts')
        # Call the assume_role method of the STSConnection object and pass the role
        # ARN and a role session name.
        assumed_role_object=sts_client.assume_role(
        RoleArn=self.assume_role_arn,
        RoleSessionName="AssumeRoleSession1")
        credentials=assumed_role_object['Credentials']
        
        scale_client = boto3.client('application-autoscaling', region_name=self.region_name, aws_access_key_id=credentials['AccessKeyId'],
        aws_secret_access_key=credentials['SecretAccessKey'],
        aws_session_token=credentials['SessionToken'],)
        return scale_client                  

    def scalable_endpoint(self):
        """Quick check to see if a endpoint already has the scalable configuration

        Returns:
            _type_: True - if endpoint already has the scalable configuration, False otherwise
        """        
        try:
            response = self.client.describe_scalable_targets(
                ServiceNamespace=self.service_name,
                ResourceIds=[self.ResourceId,],
                ScalableDimension=Scalable_Dimension) 
            logger.info("describe_scalable_targets")
            logger.info(response)    
            if len(response['ScalableTargets']) == 0:
                logger.info("endpoint does not has scalable config") 
                return False
            else: 
                logger.info("endpoint has a scalable config") 
                return True       
        except ClientError as err:
                logger.error(
                    "Couldn't get endpoints. Here's why: %s: %s",
                    err.response['Error']['Code'], err.response['Error']['Message'])
                raise 

In [0]:
import mlflow.sagemaker
import boto3
import time
import mlflow
from mlflow.deployments import get_deploy_client
from mlflow.exceptions import MlflowException
import botocore
# import inference.deployment as dp
# from inference.autoscaling import AutoScalingWrapper
import uuid
import pprint

In [0]:
deployer = ModelDeployer(target_environment=target_environment,
                         deployment_name=deployment_name,
                            variant_name=variant_name,
                            instance_type=instance_type,
                            instance_count=instance_count,
                            containerStartupHealthCheckTimeoutInSeconds=containerStartupHealthCheckTimeoutInSeconds
                            )

registry_uri: databricks-uc
